In [11]:
# -*- coding: utf-8 -*-
"""
Train XGBoost model to predict Occupancy (number of people) from CO₂ readings.
Includes lag, rolling, and time features.
Saves model as .h5 (for Python) and .json (for web use).
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
import joblib
import warnings
warnings.filterwarnings("ignore")

# ========== 1) Load datasets ==========
local_path = "dataset (1).csv"
uci_train = "datatraining.txt"
uci_test1 = "datatest.txt"
uci_test2 = "datatest2.txt"

local_df = pd.read_csv(local_path)
if "epoch" in local_df.columns:
    local_df["datetime"] = pd.to_datetime(local_df["epoch"], unit="s")
elif "date" in local_df.columns:
    local_df["datetime"] = pd.to_datetime(local_df["date"])
if "co2" in local_df.columns and "CO2" not in local_df.columns:
    local_df = local_df.rename(columns={"co2": "CO2"})
if "count" in local_df.columns:
    local_df = local_df.rename(columns={"count": "Occupancy"})

cols_local_keep = [c for c in ["datetime", "CO2", "Occupancy"] if c in local_df.columns]
local_df = local_df[cols_local_keep].copy()

df_train = pd.read_csv(uci_train)
df_test1 = pd.read_csv(uci_test1)
df_test2 = pd.read_csv(uci_test2)
uci_df = pd.concat([df_train, df_test1, df_test2], ignore_index=True)

if "date" in uci_df.columns:
    uci_df["datetime"] = pd.to_datetime(uci_df["date"])
cols_uci_keep = [c for c in ["datetime", "CO2", "Occupancy"] if c in uci_df.columns]
uci_df = uci_df[cols_uci_keep].copy()

combined = pd.concat([uci_df, local_df], ignore_index=True)
combined = combined.sort_values("datetime").reset_index(drop=True)
combined = combined.dropna(subset=["CO2", "Occupancy"])

print(f"Combined rows: {combined.shape[0]}")

# ========== 2) Feature Engineering ==========
df = combined.copy()
df["co2"] = df["CO2"].astype(float)
df = df.sort_values("datetime").reset_index(drop=True)

# --- Lag features ---
for lag in (1, 2, 3, 5, 10):
    df[f"co2_lag_{lag}"] = df["co2"].shift(lag)

# --- Rolling mean/std ---
df["co2_roll_mean_5"] = df["co2"].rolling(window=5, min_periods=1).mean().shift(1)
df["co2_roll_std_5"] = df["co2"].rolling(window=5, min_periods=1).std().fillna(0).shift(1)

# --- Diff / pct change ---
df["co2_diff_1"] = df["co2"].diff(1).fillna(0)
df["co2_pct_change_1"] = df["co2"].pct_change(1).fillna(0)

# --- Time-based rolling mean ---
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.set_index("datetime")
df["co2_roll_1min"] = df["co2"].rolling("1T", min_periods=1).mean().shift(1)
df["co2_roll_3min"] = df["co2"].rolling("3T", min_periods=1).mean().shift(1)
df["co2_roll_5min"] = df["co2"].rolling("5T", min_periods=1).mean().shift(1)
df["co2_roll_10min"] = df["co2"].rolling("10T", min_periods=1).mean().shift(1)
df = df.reset_index()

# --- Time feature ---
df["hour"] = df["datetime"].dt.hour

df = df.dropna().reset_index(drop=True)

feature_cols = [
    "co2",
    "co2_lag_1", "co2_lag_3", "co2_lag_5",
    "co2_roll_mean_5", "co2_roll_std_5",
    "co2_diff_1", "co2_pct_change_1",
    "hour",
    "co2_roll_1min", "co2_roll_3min", "co2_roll_5min", "co2_roll_10min"
]
feature_cols = [c for c in feature_cols if c in df.columns]

X = df[feature_cols].copy()
y = df["Occupancy"].astype(float).copy()

print("Features used:", feature_cols)
print("Total samples after FE:", X.shape[0])

# ========== 3) Train/Test Split ==========
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, shuffle=True
)

# ========== 4) Model Training (XGBoost) ==========
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = xgb.XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric="rmse"
)

r2_scores, rmse_scores = [], []
for train_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", model)
    ])
    pipe.fit(X_tr, y_tr)
    y_pred = pipe.predict(X_val)
    r2_scores.append(r2_score(y_val, y_pred))
    rmse_scores.append(np.sqrt(mean_squared_error(y_val, y_pred)))

print(f"\nXGBoost CV: R²={np.mean(r2_scores):.4f}, RMSE={np.mean(rmse_scores):.4f}")

# ========== 5) Train Final Model ==========
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", model)
])
pipeline.fit(X_train, y_train)

y_pred_test = pipeline.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"Test R2 = {test_r2:.4f}, RMSE = {test_rmse:.4f}")

# ========== 6) Save model (.h5 and .json) ==========
out_path = "occupancy_from_co2_xgb_model.h5"
joblib.dump({"pipeline": pipeline, "feature_cols": feature_cols}, out_path)
print(f"Model saved to: {out_path}")

# --- Save XGBoost to .json (for web use) ---
pipeline.named_steps["model"].save_model("occupancy_from_co2_xgb_model.json")
print("Exported model to JSON file: occupancy_from_co2_xgb_model.json")

# ========== 7) Helper function ==========
def predict_from_history(co2_history, datetime_obj):
    hist = list(co2_history)
    while len(hist) < 10:
        hist.insert(0, hist[0])
    hist = np.array(hist)
    latest = float(hist[-1])
    lag1 = float(hist[-2])
    lag3 = float(hist[-4])
    lag5 = float(hist[-6])
    roll5 = np.mean(hist[-5:])
    rollstd5 = np.std(hist[-5:])
    diff1 = latest - lag1
    pct1 = (diff1 / lag1) if lag1 != 0 else 0.0

    # --- ใหม่: rolling mean ระยะสั้นเพิ่มเติม ---
    roll_1min = np.mean(hist[-2:])   # 1 นาทีล่าสุด (2 ค่า)
    roll_3min = np.mean(hist[-4:])   # 3 นาทีล่าสุด (4 ค่า)
    roll_5min = np.mean(hist[-6:])   # 5 นาทีล่าสุด (6 ค่า)
    roll_10min = np.mean(hist[-10:]) # 10 นาทีล่าสุด (10 ค่า)

    hour = datetime_obj.hour

    feat_dict = {
        "co2": latest,
        "co2_lag_1": lag1,
        "co2_lag_3": lag3,
        "co2_lag_5": lag5,
        "co2_roll_mean_5": roll5,
        "co2_roll_std_5": rollstd5,
        "co2_diff_1": diff1,
        "co2_pct_change_1": pct1,
        "hour": hour,
        "co2_roll_1min": roll_1min,
        "co2_roll_3min": roll_3min,
        "co2_roll_5min": roll_5min,
        "co2_roll_10min": roll_10min
    }

    model_obj = joblib.load(out_path)
    pipeline_loaded = model_obj["pipeline"]
    feature_cols_model = model_obj["feature_cols"]
    feat = [feat_dict[c] for c in feature_cols_model if c in feat_dict]
    pred = pipeline_loaded.predict([feat])[0]
    return pred

# ตัวอย่างทดสอบ
# from datetime import datetime
# print(predict_from_history([800, 810, 820, 830, 840, 850, 860, 870, 880, 890], datetime.now()))


Combined rows: 26861
Features used: ['co2', 'co2_lag_1', 'co2_lag_3', 'co2_lag_5', 'co2_roll_mean_5', 'co2_roll_std_5', 'co2_diff_1', 'co2_pct_change_1', 'hour', 'co2_roll_1min', 'co2_roll_3min', 'co2_roll_5min', 'co2_roll_10min']
Total samples after FE: 26851

XGBoost CV: R²=0.9280, RMSE=1.0824
Test R2 = 0.9314, RMSE = 1.0300
Model saved to: occupancy_from_co2_xgb_model.h5
Exported model to JSON file: occupancy_from_co2_xgb_model.json


In [12]:
df

,datetime,CO2,Occupancy,co2,co2_lag_1,co2_lag_2,co2_lag_3,co2_lag_5,co2_lag_10,co2_roll_mean_5,co2_roll_std_5,co2_diff_1,co2_pct_change_1,co2_roll_1min,co2_roll_3min,co2_roll_5min,co2_roll_10min,hour
0,2015-02-02 14:29:00,815.250000,1,815.250000,809.000000,803.20,797.00,790.0,749.200000,799.440000,7.118146,6.250000,0.007726,809.000000,803.066667,799.440000,783.021667,14
1,2015-02-02 14:30:00,824.000000,1,824.000000,815.250000,809.00,803.20,798.0,760.400000,804.490000,7.681178,8.750000,0.010733,815.250000,809.150000,804.490000,789.626667,14
2,2015-02-02 14:31:00,832.000000,1,832.000000,824.000000,815.25,809.00,797.0,769.666667,809.690000,10.480005,8.000000,0.009709,824.000000,816.083333,809.690000,795.986667,14
3,2015-02-02 14:31:59,845.333333,1,845.333333,832.000000,824.00,815.25,803.2,774.750000,816.690000,11.521957,13.333333,0.016026,832.000000,823.750000,816.690000,802.220000,14
4,2015-02-02 14:32:59,852.400000,1,852.400000,845.333333,832.00,824.00,809.0,779.000000,825.116667,14.269188,7.066667,0.008360,838.666667,829.145833,825.116667,806.139394,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26846,2025-10-02 17:45:20,2388.000000,14,2388.000000,2390.000000,2384.00,2387.00,2389.0,2387.000000,2387.400000,2.302173,-2.000000,-0.000837,2387.000000,2387.500000,2387.600000,2390.950000,17
26847,2025-10-02 17:45:50,2394.000000,14,2394.000000,2388.000000,2390.00,2384.00,2387.0,2383.000000,2387.200000,2.167948,6.000000,0.002513,2389.000000,2387.500000,2387.700000,2390.750000,17
26848,2025-10-02 17:46:20,2399.000000,14,2399.000000,2394.000000,2388.00,2390.00,2387.0,2392.000000,2388.600000,3.714835,5.000000,0.002089,2391.000000,2388.333333,2388.800000,2391.000000,17
26849,2025-10-02 17:46:50,2407.000000,14,2407.000000,2399.000000,2394.00,2388.00,2384.0,2389.000000,2391.000000,5.744563,8.000000,0.003335,2396.500000,2390.333333,2389.500000,2391.800000,17
